In [1]:
import io
import torch
import pandas as pd

from pathlib import Path
import os
home = str(Path.home())

from PIL import Image
import requests
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import torch.nn as nn

from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, datasets
from torchvision.datasets import ImageNet

from ud188 import helper
from data_prep import get_loaders
from dog_breeds import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

First let's get unnormalized scores and then compute loss using 3 methods:

- manually;
- using `CrossEntropyLoss`;
- using `LogSoftmax` and `NLLLoss`;

In [6]:
torch.manual_seed(42)

In [7]:
loaders = get_loaders(batch_size=3)
model = get_model()

In [8]:
train_loader = loaders['train']

In [9]:
model.train();

In [11]:
data, target = next(iter(train_loader))

In [14]:
data.shape, target

(torch.Size([3, 3, 224, 224]), tensor([ 45, 102,  23]))

In [12]:
output = model(data)

In [15]:
output.shape

torch.Size([3, 133])

### `CrossEntropyLoss`

In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
loss_cross_entropy = criterion(output, target)

In [18]:
loss_cross_entropy.item()

4.874864101409912

### `LogSoftmax` and `NLLLoss`

In [19]:
output_log_softmax = nn.LogSoftmax(dim=1)(output)

In [20]:
criterion2 = nn.NLLLoss()

In [23]:
loss_nllloss = criterion2(output_log_softmax, target)

In [24]:
loss_nllloss.item()

4.874864101409912

So the first 2 methods indeed give the same result. Now let's try to get this number manually.

### manually

In [25]:
output_softmax = torch.softmax(output, dim=1)

In [26]:
output.shape

torch.Size([3, 133])

In [27]:
output_softmax.shape

torch.Size([3, 133])

In [28]:
torch.sum(output_softmax, dim=1)

tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)

In [29]:
target

tensor([ 45, 102,  23])

In [34]:
output_softmax_target = output_softmax[range(len(target)), target]
output_softmax_target

tensor([0.0060, 0.0079, 0.0095], grad_fn=<IndexBackward>)

In [33]:
output_softmax[0, 45], output_softmax[1, 102], output_softmax[2, 23]

(tensor(0.0060, grad_fn=<SelectBackward>),
 tensor(0.0079, grad_fn=<SelectBackward>),
 tensor(0.0095, grad_fn=<SelectBackward>))

In [37]:
torch.mean(-torch.log(output_softmax_target)).item()

4.874864101409912

And here we go - yet again the same number.